In [ ]:
import cv2
import numpy as np
import yaml


def undistort_mei_from_yaml(img, yaml_file):
    """
    使用 MEI 模型的 YAML 文件对鱼眼图像去畸变

    参数:
        img (np.ndarray): 原始鱼眼图像
        yaml_file (str): CamOdoCal 导出的相机标定 YAML 文件路径

    返回:
        undistorted (np.ndarray): 去畸变后的图像
    """
    # 读取 YAML 文件
    # with open(yaml_file, "r") as f:
    #     data = yaml.safe_load(f)
    with open(yaml_file, "r") as f:
        lines = f.readlines()

    # 去掉以 %YAML 开头的行
    lines = [line for line in lines if not line.startswith("%")]

    data = yaml.safe_load("".join(lines))
    # 提取参数
    fx = data["projection_parameters"]["gamma1"]
    fy = data["projection_parameters"]["gamma2"]
    cx = data["projection_parameters"]["u0"]
    cy = data["projection_parameters"]["v0"]
    xi = data["mirror_parameters"]["xi"]
    k1 = data["distortion_parameters"].get("k1", 0)
    k2 = data["distortion_parameters"].get("k2", 0)
    p1 = data["distortion_parameters"].get("p1", 0)
    p2 = data["distortion_parameters"].get("p2", 0)

    h, w = img.shape[:2]

    # 构建去畸变映射
    map_x = np.zeros((h, w), dtype=np.float32)
    map_y = np.zeros((h, w), dtype=np.float32)

    for v in range(h):
        for u in range(w):
            # 归一化像素坐标
            x = (u - cx) / fx
            y = (v - cy) / fy

            r2 = x * x + y * y
            denom = xi + np.sqrt(1 + (1 - xi * xi) * r2)
            X = x
            Y = y
            Z = 1 - xi * (r2 + 1) / denom

            # 单位球面归一化
            norm = np.sqrt(X * X + Y * Y + Z * Z)
            X /= norm
            Y /= norm
            Z /= norm

            # 应用径向 + 切向畸变
            r2 = X * X + Y * Y
            x_dist = (
                X * (1 + k1 * r2 + k2 * r2 * r2)
                + 2 * p1 * X * Y
                + p2 * (r2 + 2 * X * X)
            )
            y_dist = (
                Y * (1 + k1 * r2 + k2 * r2 * r2)
                + p1 * (r2 + 2 * Y * Y)
                + 2 * p2 * X * Y
            )

            # 投影回原图像像素
            map_x[v, u] = fx * x_dist + cx
            map_y[v, u] = fy * y_dist + cy

    # 去畸变
    undistorted = cv2.remap(img, map_x, map_y, cv2.INTER_LINEAR)
    return undistorted


# -------------------
# 使用示例
# -------------------
img = cv2.imread(
    "/home/ubuntu/Desktop/tmp/camodocal_data/back_fisheye_808/1758006446_327292793_sensor_msgs__msg__Image.jpg"
)
undistorted_img = undistort_mei_from_yaml(
    img, "/home/ubuntu/Desktop/tmp/camodocal_data/back_fisheye_camera_calib.yaml"
)
cv2.imwrite("test_undistorted.jpg", undistorted_img)


/tmp/ipykernel_41206/220538393.py:50: RuntimeWarning: invalid value encountered in sqrt
  denom = xi + np.sqrt(1 + (1 - xi*xi) * r2)


True

In [5]:
import cv2
import numpy as np
import yaml


def undistort_mei_image(img, yaml_file, new_size=None):
    """
    使用 CamOdoCal 的 MEI YAML 文件，对鱼眼图像进行去畸变

    参数:
        img (np.ndarray): 原始鱼眼图像
        yaml_file (str): CamOdoCal 导出的 YAML 文件路径
        new_size (tuple, optional): 去畸变图像大小 (width, height)，默认使用原图大小

    返回:
        undistorted (np.ndarray): 去畸变后的图像
    """
    # 读取 YAML 文件
    with open(yaml_file, "r") as f:
        lines = f.readlines()
        # 去掉 %YAML:1.0 指令行
        lines = [line for line in lines if not line.startswith("%")]
        data = yaml.safe_load("".join(lines))

    # 内参矩阵 K
    fx = data["projection_parameters"]["gamma1"]
    fy = data["projection_parameters"]["gamma2"]
    cx = data["projection_parameters"]["u0"]
    cy = data["projection_parameters"]["v0"]
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=np.float64)

    # 畸变参数 D
    k1 = data["distortion_parameters"].get("k1", 0.0)
    k2 = data["distortion_parameters"].get("k2", 0.0)
    p1 = data["distortion_parameters"].get("p1", 0.0)
    p2 = data["distortion_parameters"].get("p2", 0.0)
    D = np.array([k1, k2, p1, p2], dtype=np.float64)

    # 镜面参数 xi
    xi = np.array([float(data['mirror_parameters']['xi'])], dtype=np.float64)

    # 设置去畸变图像大小
    if new_size is None:
        new_size = (img.shape[1], img.shape[0])  # (width, height)

    # 去畸变
    # undistorted = cv2.omnidir.undistortImage(
    #     img, K, D, xi, flags=cv2.omnidir.RECTIFY_PERSPECTIVE, new_size=new_size
    # )
    undistorted = cv2.omnidir.undistortImage(
    img,
    K,
    D,
    xi,
    flags=cv2.omnidir.RECTIFY_CYLINDRICAL,  # 或 RECTIFY_LONGLATI
    new_size=new_size
)

    return undistorted
# -------------------
img = cv2.imread(
    "1758006449_320193644_sensor_msgs__msg__Image.jpg"
)
undistorted_img = undistort_mei_image(
    img, "/home/ubuntu/Desktop/tmp/camodocal_data/back_fisheye_camera_calib.yaml"
)
cv2.imwrite("test2_undistorted.jpg", undistorted_img)


True

In [6]:
import cv2
import numpy as np
import yaml

def undistort_mei_image_remap(img, yaml_file, new_size=None, rectify_mode='perspective'):
    """
    使用 initUndistortRectifyMap 对鱼眼/MEI 图像去畸变
    支持 perspective, cylindrical, lonlat 三种模式

    参数:
        img (np.ndarray): 原始鱼眼图像
        yaml_file (str): CamOdoCal 导出的 YAML 文件路径
        new_size (tuple, optional): 输出图像大小 (width, height)，默认原图大小
        rectify_mode (str): 'perspective' | 'cylindrical' | 'lonlat'

    返回:
        undistorted (np.ndarray): 去畸变图像
    """
    # 读取 YAML
    with open(yaml_file, 'r') as f:
        lines = [line for line in f.readlines() if not line.startswith('%')]
        data = yaml.safe_load("".join(lines))

    # 相机内参
    fx = data['projection_parameters']['gamma1']
    fy = data['projection_parameters']['gamma2']
    cx = data['projection_parameters']['u0']
    cy = data['projection_parameters']['v0']
    K = np.array([[fx, 0, cx],
                  [0, fy, cy],
                  [0, 0, 1]], dtype=np.float64)

    # 畸变系数
    k1 = data['distortion_parameters'].get('k1', 0.0)
    k2 = data['distortion_parameters'].get('k2', 0.0)
    p1 = data['distortion_parameters'].get('p1', 0.0)
    p2 = data['distortion_parameters'].get('p2', 0.0)
    D = np.array([k1, k2, p1, p2], dtype=np.float64)

    # 镜面参数 xi
    xi = np.array([float(data['mirror_parameters']['xi'])], dtype=np.float64)

    # 输出尺寸
    if new_size is None:
        new_size = (img.shape[1], img.shape[0])  # (width, height)

    # rectification 类型
    mode_dict = {
        'perspective': cv2.omnidir.RECTIFY_PERSPECTIVE,
        'cylindrical': cv2.omnidir.RECTIFY_CYLINDRICAL,
        'lonlat': cv2.omnidir.RECTIFY_LONGLATI
    }
    flag = mode_dict.get(rectify_mode.lower(), cv2.omnidir.RECTIFY_PERSPECTIVE)

    # 生成映射表
    map1, map2 = cv2.omnidir.initUndistortRectifyMap(
        K, D, xi,
        R=np.eye(3, dtype=np.float64),
        P=K,
        size=new_size,
        m1type=cv2.CV_32FC1,
        flags=flag
    )

    # 去畸变
    undistorted = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR)

    return undistorted
img = cv2.imread("1758006449_320193644_sensor_msgs__msg__Image.jpg")

# perspective 透视去畸变
undistorted_p = undistort_mei_image_remap(img, "/home/ubuntu/Desktop/tmp/camodocal_data/back_fisheye_camera_calib.yaml", rectify_mode='perspective')

# cylindrical 柱面投影
undistorted_c = undistort_mei_image_remap(img, "/home/ubuntu/Desktop/tmp/camodocal_data/back_fisheye_camera_calib.yaml", rectify_mode='cylindrical')

# 保存结果
cv2.imwrite("test_undistorted_p.jpg", undistorted_p)
cv2.imwrite("test_undistorted_c.jpg", undistorted_c)


True